# BepiColombo SPICE Kernel Dataset Status
SKD version: {skd_version} <br>
Creation: {current_time} by ESA SPICE Service (ESAC/ESA). <br>
Bepi Test specification: SPICE-CROSS-TN-190322 <br>


## Startup
Prepare the testing environment at first stage, so if any validation step fails stopping the execution, the tests are already defined and in fail state.

In [4]:
from spival.classes.history import TestHistory

test_history = TestHistory()
test_history.add_test('XM-C1', 'Metakernel is valid', 'Consistency')
test_history.add_test('XM-V5', 'OEM to SPK error', 'Validity/Error', threshold='~100m')
test_history.add_test('XM-V4', 'Measured Quaternions to CK error', 'Validity/Warning', threshold='5 mdeg')
test_history.add_test('XM-Q3', 'Attitude Error', 'Quality/Data')
test_history.add_test('XM-Q4', 'MPO Orientation (quaternions w.r.. J2000)', 'Quality/Data')
test_history.add_test('XM-Q5-MPO', 'MPO Solar Array (SA) Angles', 'Quality/Data')
test_history.add_test('XM-Q5-MTM', 'MTM Solar Array (SA) Angles', 'Quality/Data')
test_history.add_test('XM-Q1-MPO', 'MPO SA Solar Aspect Angle', 'Quality/Data')
test_history.add_test('XM-Q1-MTM', 'MTM SA Solar Aspect Angle', 'Quality/Data')
test_history.add_test('XM-Q6', 'MPO High Gain Antenna Angles', 'Quality/Data')
test_history.add_test('XM-Q7', 'MPO High Gain Antenna - Earth Angle', 'Quality/Data')
test_history.add_test('XM-V1', 'S/C Clock Drift', 'Validity/Error')

### XM-C1 - Test Metakernel is valid 
The metakernel is loaded, the scenario covers a week with a finish time set by the end of coverage of the latest Measured Attitude Kernel.

In [ ]:
from spiops import spiops

spiops.load('{metakernel}')               

test_history.set_test_result('XM-C1', True)

start_time = '{start_time_measured}'                                      # Start time
finish_time ='{finish_time_measured}'  

interval = spiops.TimeWindow(start_time, finish_time,resolution=60) # spiops object TimeWindow generated
sun = spiops.Target('SUN', time=interval, frame='IAU_SUN')       # spiops object Target Sun generated
mpo = spiops.Observer('MPO', time=interval, target=sun)            # spiops object Observer MPO generated
mtm = spiops.Observer('MTM', time=interval, target=sun)            # spiops object Observer MTM generated

### <center><span style='color:Green'> Metakernel loaded successfully.  </span></center>
## Coverage

The coverage provided by the SPK and CK files is displayed for the Operational and the Planning meta-kernels. 

The Data type designation includes a reference to the originator of the data, the type of data and the reference period is provided. This is a three letter acronym. The first letter defines the file originator:
*    s: Science Operation
*    m: Mission Analysis
*    f: Flight Dynamic

the second letter defines the reference period of the data:
*    c: Cruise phase reference
*    l: Science phase Long term reference
*    m: Science phase Medium term reference
*    s: Science phase Short term reference
*    o: Undefined reference

the third letter indicates the type of data
*    <span style='color:Orange'> p: Predicted data</span>
*    <span style='color:Green'> r: Reconstructed data</span>
*    <span style='color:Red'> t: Test data</span>
*    <span style='color:Purple'> c: Commanded data (from Housekeeping Telemetry)</span>
*    <span style='color:Blue'> m: Measured data (from Housekeeping Telemetry)</span>


In [ ]:
spiops.ck_coverage_timeline('{metakernel}', ['MPO_SPACECRAFT', 'MMO_SPACECRAFT', 'MTM_SPACECRAFT'], plot_height=500)
spiops.ck_coverage_timeline('{metakernel}'.replace('ops', 'plan'), ['MPO_SPACECRAFT', 'MMO_SPACECRAFT', 'MTM_SPACECRAFT'], plot_height=400)

In [ ]:
spiops.spk_coverage_timeline('{metakernel}', ['MPO', 'MMO', 'MTM'], plot_height=200)
spiops.spk_coverage_timeline('{metakernel}'.replace('ops', 'plan'), ['MPO', 'MMO', 'MTM'], plot_height=400)

### XM-V5 - OEM to SPK error
Comparison of spacecraft position and velocity between source OEM and generated SPK 

In [ ]:
target_spk = '{skd_path}/spk/{reconstructed_spk}'
max_pos_err, max_vel_err = spiops.spkVsOem('MPO', target_spk, plot_style='line', notebook=True)
test_history.set_test_result('XM-V5', (max_pos_err != None) and (max_pos_err < 0.1))

### XM-V4 - Measured Quaternions to CK error
Comparison of spacecraft orientation (quaternion) between source AOCS Measured Quaternions and generated CK 

In [ ]:
target_ck = '{skd_path}/ck/{measured_ck}'
max_err = spiops.ckVsAocs('MPO', target_ck, plot_style='line', notebook=True)
test_history.set_test_result('XM-V4', (max_err != None) and (max_err < 5)) # 5mdeg

### XM-Q3 - Attitude Error
Comparison of +Z axis orientation between predicted and measured attitude in arcseconds 

In [ ]:
measured_ck = '{skd_path}/ck/{measured_ck}'
predicted_ck = '{skd_path}/ck/{predicted_ck}'
resolution = 4

spiops.ckdiff_error(measured_ck, predicted_ck, 'MPO_SPACECRAFT', 'J2000', resolution, 0.001, 
                              plot_style='circle', utc_start=start_time, utc_finish=finish_time, notebook=True)
test_history.set_test_result('XM-Q3', True)

## S/C Structures Orientation
Quaternions, MTM and MPO Solar Array Orientation and Solar Aspect Angle, High Gain Antenna Orientation and HGA boresight-Earth Angle.

### XM-Q4 - MPO Orientation (quaternions w.r.. J2000)

In [ ]:
mpo.Plot('quaternions', notebook=True)         
test_history.set_test_result('XM-Q4', True)

### XM-Q5-MPO - MPO Solar Array (SA) Angles 

In [ ]:
mpo.Plot('sa_ang', notebook=True)              
test_history.set_test_result('XM-Q5-MPO', True)

### XM-Q5-MTM - MTM Solar Array (SA) Angles

In [ ]:
mtm.Plot('sa_ang', notebook=True)              
test_history.set_test_result('XM-Q5-MTM', True)

### XM-Q1-MPO - MPO SA Solar Aspect Angle

In [ ]:
mpo.Plot('saa_sa', notebook=True)              
test_history.set_test_result('XM-Q1-MPO', True)

### XM-Q1-MTM - MTM SA Solar Aspect Angle

In [ ]:
mtm.Plot('saa_sa', notebook=True)              
test_history.set_test_result('XM-Q1-MTM', True)

### XM-Q6 - MPO High Gain Antenna Angles

In [ ]:
mpo.Plot('hga_angles', notebook=True)                 
test_history.set_test_result('XM-Q6', True)       

### XM-Q7- MPO High Gain Antenna - Earth Angle

In [ ]:
mpo.Plot('hga_earth', notebook=True)
test_history.set_test_result('XM-Q7', True)

### XM-V1 - S/C Clock Drift 

The following plot shows the drift of the S/C Clock.

In [ ]:
mpo.Plot('clock_drift', notebook=True)
test_history.set_test_result('XM-V1', True)

In [ ]:
# Show validation results
test_history.show_tests()